# Encontrando APIs ocultas

Cuando una compañia desarrolla un sitio web muchas veces separa lo que se dice el _frontend_, que es la parte visible del sitio y la cual se ejecuta en tu navegador, del _backend_, la parte del sitio que realiza el computo mas pesado y se ejecuta en servidores/computadoras de la compañia.

Para comunicar el _backend_ con el _frontend_ una forma popular es desarrollar REST APIs, a veces estas son públicas pero a veces están ocultas y las utilizamos sin darnos cuenta cuando interactuamos con un sitio web.

Ejemplo de un sitio web y su API: http://numbersapi.com/

Hoy veremos como encontrarlas y utilizarlas para extraer datos de forma cómoda

## Ejemplo 1: Valor del Bitcoin

Para este primer ejemplo veremos como encontrar una API oculta en https://coinmarketcap.com/es

Luego veremos como utilizarla de 3 maneras:
1. Desde el navegador
2. Con cURL (Un programa de linea de comandos para transferir datos de internet)
3. Con Python

[Herramienta](https://www.epochconverter.com/)

### Método utilizando python

Usaremos el sitio web de [coinmarketcap](https://coinmarketcap.com/es/currencies/bitcoin/) que proporciona datos del Bitcoin en tiempo real.

#### Pasos para encontrar la API oculta y utilizarla con Python

1. Entrar al sitio web:

    - Visita el sitio

2. Abrir las herramientas de desarrollo:

    - Presiona Ctrl+Shift+I para abrir las herramientas de desarrollo en tu navegador.
    - Ve a la solapa Network y selecciona XHR.

3. Recargar el sitio y analizar las peticiones:

    - Recarga la página para capturar todas las solicitudes que realiza el sitio.
    - En la pestaña XHR, revisa las peticiones que se están realizando. Busca aquellas que tienen nombres relevantes o cuyo Response contiene datos que te interesen, como el valor del Bitcoin.

4. Identificar la API y copiar el comando cURL:

    - Una vez que encuentres una solicitud relevante, haz clic derecho sobre ella y selecciona "Copy as cURL(bash)" para copiar el comando cURL que realiza la solicitud.

5. Convertir el comando cURL a código Python:

    - Utiliza [curlconverter.com](https://curlconverter.com/) para convertir el comando cURL a código Python. Pega el cURL copiado y el sitio te generará un código en Python utilizando la librería requests.

6. Escribir el código en Python:

    - A partir del código generado, puedes realizar la solicitud a la API y extraer los datos que necesites.

In [1]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'es-ES,es;q=0.8',
    'cache-control': 'no-cache',
    'origin': 'https://coinmarketcap.com',
    'platform': 'web',
    'priority': 'u=1, i',
    'referer': 'https://coinmarketcap.com/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'x-request-id': 'ae6571a8c35944b8863462441f81f1c9',
}

params = {
    'id': '1',
    'range': '1D',
}

response = requests.get('https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart', params=params, headers=headers)
response.json()

{'data': {'points': {'1737972900': {'v': [99153.18570395232,
     53695688024.48,
     1964791269251.5935,
     1,
     19815715.0],
    'c': [99153.18570395232, 53695688024.48, 1964791269251.5935]},
   '1737973200': {'v': [99021.43163707847,
     53852660967.17,
     1962180468212.3303,
     1,
     19815715.0],
    'c': [99021.43163707847, 53852660967.17, 1962180468212.3303]},
   '1737973500': {'v': [99138.23882076601,
     53966596620.4,
     1964495086074.2354,
     1,
     19815715.0],
    'c': [99138.23882076601, 53966596620.4, 1964495086074.2354]},
   '1737973800': {'v': [99088.94779158047,
     54108978168.79,
     1963518349087.838,
     1,
     19815715.0],
    'c': [99088.94779158047, 54108978168.79, 1963518349087.838]},
   '1737974100': {'v': [98797.38841351782,
     54228954263.26,
     1957740891546.5713,
     1,
     19815715.0],
    'c': [98797.38841351782, 54228954263.26, 1957740891546.5713]},
   '1737974400': {'v': [98779.22804424787,
     54457435421.76,
     1957381

## Mejorando el código

Analizando la API se puede ver que tiene al menos un parámetro modificables, el periodo.

Podemos poner todo dentro de una función que extraiga los precios del Bitcoin para un periodo determinado. De esta forma será más fácil de usar y no necesitemos repetir el código.

In [2]:
import requests

headers = {
    'accept': 'application/json, text/plain, */*',
    'accept-language': 'es-ES,es;q=0.8',
    'cache-control': 'no-cache',
    'origin': 'https://coinmarketcap.com',
    'platform': 'web',
    'priority': 'u=1, i',
    'referer': 'https://coinmarketcap.com/',
    'sec-ch-ua': '"Chromium";v="128", "Not;A=Brand";v="24", "Brave";v="128"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'same-site',
    'sec-gpc': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/128.0.0.0 Safari/537.36',
    'x-request-id': 'ae6571a8c35944b8863462441f81f1c9',
}

params = {
    'id': '1',  # ID de la criptomoneda (1 es Bitcoin)
    'range': '7D',  # Rango de tiempo para los últimos 7 días
}

response = requests.get('https://api.coinmarketcap.com/data-api/v3/cryptocurrency/detail/chart', params=params, headers=headers)

if response.status_code == 200:
    data = response.json()
    print(data)
else:
    print("Error:", response.status_code)


{'data': {'points': {'1737454500': {'v': [102578.87638404829, 105730156759.31, 2032405535684.787, 1, 19813100.0], 'c': [102578.87638404829, 105730156759.31, 2032405535684.787]}, '1737455400': {'v': [103044.0035355851, 105633199948.76, 2041621146450.9011, 1, 19813100.0], 'c': [103044.0035355851, 105633199948.76, 2041621146450.9011]}, '1737456300': {'v': [102896.06308046517, 105646533220.72, 2038690604795.9429, 1, 19813106.0], 'c': [102896.06308046517, 105646533220.72, 2038690604795.9429]}, '1737457200': {'v': [103046.20588565222, 105551387227.65, 2041665400110.2512, 1, 19813106.0], 'c': [103046.20588565222, 105551387227.65, 2041665400110.2512]}, '1737458100': {'v': [103481.37686651618, 105342287451.39, 2050287488882.233, 1, 19813106.0], 'c': [103481.37686651618, 105342287451.39, 2050287488882.233]}, '1737459000': {'v': [103559.21480101426, 105576320469.38, 2051829700129.2644, 1, 19813106.0], 'c': [103559.21480101426, 105576320469.38, 2051829700129.2644]}, '1737459900': {'v': [103801.897

In [3]:
import pandas as pd

points = data['data']['points']

# Convertir los datos en un DataFrame
df = pd.DataFrame.from_dict(points, orient='index')

# Separar los valores de 'v' en columnas
df[['price', 'volume_24h', 'market_cap', 'c4', 'c5']] = pd.DataFrame(df['v'].tolist(), index=df.index)

# Eliminar las columnas innecesarias ('v' y 'c')
df = df.drop(columns=['v', 'c4', 'c5', 'c'])

# Convertir la columna index (timestamps) a un formato de fecha legible
df.index = pd.to_datetime(pd.to_numeric(df.index), unit='s')

# Mostrar el DataFrame
df

,price,volume_24h,market_cap
2025-01-21 10:15:00,102578.876384,1.057302e+11,2.032406e+12
2025-01-21 10:30:00,103044.003536,1.056332e+11,2.041621e+12
2025-01-21 10:45:00,102896.063080,1.056465e+11,2.038691e+12
2025-01-21 11:00:00,103046.205886,1.055514e+11,2.041665e+12
2025-01-21 11:15:00,103481.376867,1.053423e+11,2.050287e+12
...,...,...,...
2025-01-28 09:10:00,102754.647814,6.833392e+10,2.036203e+12
2025-01-28 09:25:00,102609.972221,6.785535e+10,2.033336e+12
2025-01-28 09:35:00,102631.781637,6.759057e+10,2.033769e+12
2025-01-28 09:50:00,102637.182398,6.734740e+10,2.033877e+12


In [4]:
df['market_cap']/df['price']


2025-01-21 10:15:00    19813100.0
2025-01-21 10:30:00    19813100.0
2025-01-21 10:45:00    19813106.0
2025-01-21 11:00:00    19813106.0
2025-01-21 11:15:00    19813106.0
                          ...    
2025-01-28 09:10:00    19816168.0
2025-01-28 09:25:00    19816168.0
2025-01-28 09:35:00    19816168.0
2025-01-28 09:50:00    19816178.0
2025-01-28 10:12:24    19816178.0
Length: 701, dtype: float64

In [5]:
import plotly.graph_objects as go

fig = go.Figure()

# Graficar el Precio
fig.add_trace(go.Scatter(x=df.index, y=df['price'], mode='lines', name='Price (USD)', yaxis='y1'))

# Graficar el Volumen de 24h
fig.add_trace(go.Scatter(x=df.index, y=df['volume_24h'], mode='lines', name='Volume 24h (USD)', yaxis='y2'))

# Configurar los ejes con escala logarítmica
fig.update_layout(
    title='Evolución de Precio, Volumen de 24h y Capitalización de Mercado (Escala Logarítmica)',
    xaxis_title='Fecha',
    yaxis=dict(
        title='Price (USD)',
        type='log',  # Escala logarítmica
        titlefont=dict(color='blue'),
        tickfont=dict(color='blue'),
        side='left'
    ),
    yaxis2=dict(
        title='Volume 24h (USD)',
        type='log',  # Escala logarítmica
        titlefont=dict(color='orange'),
        tickfont=dict(color='orange'),
        overlaying='y',
        side='right'
    ),
)

# Mostrar la figura
fig.show()


## Ejercicio Meteorológica

Conseguir los datos de la meteorología de Madrid: https://www.bbc.com/weather/3117735

In [6]:
import requests

headers = {
    'Accept': '*/*',
    'Sec-Fetch-Site': 'cross-site',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'If-None-Match': '"5e461e52e1158e4c6c26658ca1610e17356bb3771fbbd0992ceb22822fd72cc0"',
    'Sec-Fetch-Mode': 'cors',
    'Accept-Language': 'es-ES,es;q=0.9',
    'Origin': 'https://www.bbc.com',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/18.1 Safari/605.1.15',
    'Connection': 'keep-alive',
    'Referer': 'https://www.bbc.com/',
    'Host': 'weather-broker-cdn.api.bbci.co.uk',
    'Sec-Fetch-Dest': 'empty',
}

response = requests.get('https://weather-broker-cdn.api.bbci.co.uk/en/forecast/aggregated/3117735', headers=headers)
response

<Response [200]>

#### Crear el DataFrame

In [7]:
response.json()['forecasts'][0]['summary']['report'] #para buscar los elementos

{'enhancedWeatherDescription': 'Sunny intervals and a moderate breeze',
 'gustSpeedKph': 54,
 'gustSpeedMph': 34,
 'localDate': '2025-01-28',
 'lowermaxTemperatureC': None,
 'lowermaxTemperatureF': None,
 'lowerminTemperatureC': None,
 'lowerminTemperatureF': None,
 'maxTempC': 11,
 'maxTempF': 51,
 'minTempC': 2,
 'minTempF': 36,
 'mostLikelyHighTemperatureC': 12,
 'mostLikelyHighTemperatureF': 54,
 'mostLikelyLowTemperatureC': 0,
 'mostLikelyLowTemperatureF': 32,
 'pollenIndex': None,
 'pollenIndexBand': None,
 'pollenIndexIconText': None,
 'pollenIndexText': None,
 'pollutionIndex': None,
 'pollutionIndexBand': None,
 'pollutionIndexIconText': None,
 'pollutionIndexText': None,
 'precipitationProbabilityInPercent': 23,
 'precipitationProbabilityText': 'Low chance of precipitation',
 'sunrise': '08:28',
 'sunset': '18:28',
 'uppermaxTemperatureC': None,
 'uppermaxTemperatureF': None,
 'upperminTemperatureC': None,
 'upperminTemperatureF': None,
 'uvIndex': 3,
 'uvIndexBand': 'MODERAT

In [8]:
forecasts=response.json()['forecasts']
for forecast in forecasts:
    summary=forecast['summary']
    report= summary['report']
    record={'date:report'}
    

#### Trazar un gráfico